In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
print('-' * 80)
print('train')
train = import_data('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')

print('-' * 80)
print('test')
test = import_data('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train['playersjoined']=train.groupby('matchId')['matchId'].transform('count')
train.head()

In [ ]:
train['traveldistance']=train['walkDistance']+train['swimDistance']+train['rideDistance']
train['healsandboosts']=train['heals']+train['boosts']

## outliers

In [ ]:
train['killingwithoutmoving']=(train['kills']>0) & (train['traveldistance']== 0)
train.drop(train[train['killingwithoutmoving']==True].index,inplace=True)
train[train['winPlacePerc'].isnull()]

In [ ]:
train.drop(2744604,inplace=True)
train[train['winPlacePerc'].isnull()]

In [ ]:
train.shape

In [ ]:
train[train['walkDistance']>10000]

In [ ]:
train.drop(train[train['walkDistance']>10000].index,inplace=True)
train[train['swimDistance']>2000]

In [ ]:
train.drop(train[train['swimDistance']>2000].index,inplace=True)
train[train['rideDistance']>20000]

In [ ]:
train.drop(train[train['rideDistance']>20000].index,inplace=True)
train[train['longestKill']>1000]

In [ ]:
train.drop(train[train['longestKill']>1000].index,inplace=True)
train[train['kills']>30]

In [ ]:
train.drop(train[train['kills']>30].index,inplace=True)
train[train['roadKills']>12]

In [ ]:
train.drop(train[train['roadKills']>12].index,inplace=True)
train[train['weaponsAcquired']>70]

In [ ]:
train.drop(train[train['weaponsAcquired']>70].index,inplace=True)
train[train['heals']>35]

In [ ]:
train.drop(train[train['heals']>35].index,inplace=True)
train.shape

## Prediction modeling

In [ ]:
df=pd.concat([train,test],ignore_index=True)

In [ ]:
df.shape

In [ ]:
enc_df2 = pd.get_dummies(df[["matchType"]])
df.drop(['matchType'],axis=1,inplace=True)
#df.rename(columns={0.0:'Ungraduated',1.0:'Graduated'},inplace=True)
df = df.join(enc_df2)

df.drop(['Id','groupId','matchId'],axis=1,inplace=True)
trainx = df.iloc[:4444630]
testx = df.iloc[4444631:].fillna(0)
y=trainx["winPlacePerc"]
X = trainx.drop(["winPlacePerc"],axis=1)
X_test = testx.drop(["winPlacePerc"],axis=1)

from sklearn.model_selection import train_test_split
#xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.4,random_state=42)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

prep = MinMaxScaler()

In [ ]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
y.fillna(0,inplace=True)

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.4,random_state=42)
DTree_clf = DecisionTreeRegressor()
DTree_clf.fit(X, y)

In [ ]:
DTree_pred = DTree_clf.predict(X_test)
DTree_pred[:20]

In [ ]:
output = pd.DataFrame({'Id':test.Id,'winPlacePerc':DTree_pred})

In [ ]:
output.to_csv("submission.csv",index = False)